<a href="https://colab.research.google.com/github/graviraja/100-Days-of-NLP/blob/applications%2Fclustering/applications/clustering/covid_papers/Covid%20article%20finding%20using%20LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Drive Mounting

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
metadata_path = '/content/drive/My Drive/metadata.csv'

### Installations

In [22]:
!pip install scispacy -q

     |████████████████████████████████| 10.0MB 6.7MB/s 
     |████████████████████████████████| 13.0MB 179kB/s 
     |████████████████████████████████| 2.1MB 47.0MB/s 
     |████████████████████████████████| 296kB 46.7MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [35]:
!pip install pyldavis -q

     |████████████████████████████████| 1.6MB 3.0MB/s 
     |████████████████████████████████| 552kB 17.0MB/s 


In [24]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_sm-0.2.4.tar.gz

# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_md-0.2.4.tar.gz

     |████████████████████████████████| 70.0MB 92kB/s 
  Created wheel for en-core-sci-md: filename=en_core_sci_md-0.2.4-cp36-none-any.whl size=70498247 sha256=e691d8d51654c830c4fda816ee6f97bd7c37c54c0b6e52d0f17f19ec0baeb3de
  Stored in directory: /root/.cache/pip/wheels/12/b3/89/7fbb30f56411e8b4002eac6d5568ab46da63191a2287aa17bf
Successfully built en-core-sci-md


### Imports

In [38]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import spacy
import scispacy
import en_core_sci_sm

import pyLDAvis
import pyLDAvis.sklearn

from scipy.spatial.distance import jensenshannon

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

### Data Exploration

In [44]:
df = pd.read_csv(metadata_path)

In [45]:
df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,xqhn0vbp,1e1286db212100993d03cc22374b624f7caee956,PMC,Airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,PMC140314,12525263.0,no-cc,"BACKGROUND: Rhinovirus, the most common cause ...",2003-01-13,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",BMC Public Health,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
1,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,PMC,Discovering human history from stomach bacteria,10.1186/gb-2003-4-5-213,PMC156578,12734001.0,no-cc,Recent analyses of human pathogens have reveal...,2003-04-28,"Disotell, Todd R",Genome Biol,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
2,le0ogx1s,NaN,PMC,A new recruit for the army of the men of death,10.1186/gb-2003-4-7-113,PMC193621,12844350.0,no-cc,"The army of the men of death, in John Bunyan's...",2003-06-27,"Petsko, Gregory A",Genome Biol,NaN,NaN,False,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
3,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,PMC,Association of HLA class I with severe acute r...,10.1186/1471-2350-4-9,PMC212558,12969506.0,no-cc,BACKGROUND: The human leukocyte antigen (HLA) ...,2003-09-12,"Lin, Marie; Tseng, Hsiang-Kuang; Trejaut, Jean...",BMC Med Genet,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...
4,0qaoam29,5b68a553a7cbbea13472721cd1ad617d42b40c26,PMC,A double epidemic model for the SARS propagation,10.1186/1471-2334-3-19,PMC222908,12964944.0,no-cc,BACKGROUND: An epidemic of a Severe Acute Resp...,2003-09-10,"Ng, Tuen Wai; Turinici, Gabriel; Danchin, Antoine",BMC Infect Dis,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...


In [46]:
len(df)

52398

In [47]:
df.columns

Index(['cord_uid', 'sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'publish_time', 'authors', 'journal',
       'Microsoft Academic Paper ID', 'WHO #Covidence', 'has_pdf_parse',
       'has_pmc_xml_parse', 'full_text_file', 'url'],
      dtype='object')

In [48]:
df = df[['title', 'abstract', 'url', 'publish_time']]

In [49]:
df['publish_time'].isna().sum()

8

In [53]:
df.head()

,title,abstract,url,publish_time
0,Airborne rhinovirus detection and effect of ul...,"BACKGROUND: Rhinovirus, the most common cause ...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,2003-01-13
1,Discovering human history from stomach bacteria,Recent analyses of human pathogens have reveal...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,2003-04-28
2,A new recruit for the army of the men of death,"The army of the men of death, in John Bunyan's...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,2003-06-27
3,Association of HLA class I with severe acute r...,BACKGROUND: The human leukocyte antigen (HLA) ...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,2003-09-12
4,A double epidemic model for the SARS propagation,BACKGROUND: An epidemic of a Severe Acute Resp...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,2003-09-10


In [54]:
df = df.dropna()
df = df.reset_index(drop=True)

In [55]:
len(df)

42748

In [56]:
df['publish_year'] = df['publish_time'].apply(lambda x: int(x.split('-')[0]))

In [57]:
df.head()

,title,abstract,url,publish_time,publish_year
0,Airborne rhinovirus detection and effect of ul...,"BACKGROUND: Rhinovirus, the most common cause ...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,2003-01-13,2003
1,Discovering human history from stomach bacteria,Recent analyses of human pathogens have reveal...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,2003-04-28,2003
2,A new recruit for the army of the men of death,"The army of the men of death, in John Bunyan's...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,2003-06-27,2003
3,Association of HLA class I with severe acute r...,BACKGROUND: The human leukocyte antigen (HLA) ...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,2003-09-12,2003
4,A double epidemic model for the SARS propagation,BACKGROUND: An epidemic of a Severe Acute Resp...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,2003-09-10,2003


In [65]:
all_texts = df.abstract.values

### Pre-processing

In [63]:
nlp = en_core_sci_sm.load(disable=["tagger", "parser", "ner"])
nlp.max_length = 3000000

/usr/local/lib/python3.6/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_sci_sm' (0.2.4) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [64]:
def spacy_tokenizer(sentence):
    return [word.lemma_ for word in nlp(sentence)
            if not (word.like_num or word.is_stop or word.is_punct or word.is_space or len(word)==1)]

In [66]:
toks = spacy_tokenizer(all_texts[0])
" ".join(toks)

'BACKGROUND Rhinovirus common cause upper respiratory tract infection implicate asthma exacerbation possibly asthma death method transmission rhinovirus dispute study demonstrate aerosol transmission likely method transmission adult step study possible airborne rhinovirus transmission develope method detect aerosolized rhinovirus extend exist technology detect infectious agent nasal specimen METHODS aerosolized rhinovirus small aerosol chamber Experiments conduct decrease concentration rhinovirus determine effect UV irradiation detection rhinoviral aerosol conduct experiment expose aerosol UV dose mJ/m(2 Aerosols collect Teflon filter rhinovirus recover Qiagen AVL buff Qiagen QIAamp Viral RNA Kit Qiagen Corp. Valencia California follow semi-nested RT-PCR detection gel electrophoresis RESULTS obtain positive result filter sample collect TCID(50 aerosolized rhinovirus Ultraviolet irradiation airborne virus dose great upper-room UV germicidal irradiation application inhibit subsequent det

In [67]:
customize_stop_words = [
    'doi', 'preprint', 'copyright', 'org', 'https', 'et', 'al', 'author', 'figure', 'table',
    'rights', 'reserved', 'permission', 'use', 'used', 'using', 'biorxiv', 'medrxiv', 'license', 'fig', 'fig.', 'al.', 'Elsevier', 'PMC', 'CZI',
    '-PRON-', 'usually',
    r'\usepackage{amsbsy', r'\usepackage{amsfonts', r'\usepackage{mathrsfs', r'\usepackage{amssymb', r'\usepackage{wasysym',
    r'\setlength{\oddsidemargin}{-69pt',  r'\usepackage{upgreek', r'\documentclass[12pt]{minimal'
]

# Mark them as stop words
for w in customize_stop_words:
    nlp.vocab[w].is_stop = True

In [122]:
%%time
tfidf_vectorizer = TfidfVectorizer(tokenizer = spacy_tokenizer, min_df=10, max_features=500000)
data_vectorized = tfidf_vectorizer.fit_transform(all_texts)

CPU times: user 1min 19s, sys: 48 ms, total: 1min 19s
Wall time: 1min 19s


In [123]:
data_vectorized.shape

(42748, 15485)

### LDA Model

In [137]:
%%time
lda_model = LatentDirichletAllocation(
    n_components=35,
    max_iter=30,
    n_jobs=-1,
    random_state=42
)

lda_output = lda_model.fit_transform(data_vectorized)

CPU times: user 8.9 s, sys: 1.65 s, total: 10.5 s
Wall time: 8min 47s


In [138]:
# higher the better
print(f"Log Likelihood: {lda_model.score(data_vectorized)}")

# lower the better
print(f"preplexity: {lda_model.perplexity(data_vectorized)}")

Log Likelihood: -2758012.585355785
preplexity: 18887.989237623493


### Visualization of Topics

In [139]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, tfidf_vectorizer, mds='tsne')
vis

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
3      -53.972095   97.037666       1        1  16.071473
5       44.523598  137.011734       2        1  15.452957
24      97.307495  109.215515       3        1   9.236582
32      13.464739  193.195190       4        1   8.967898
28      -8.464339  127.240921       5        1   7.589956
30    -112.666107  125.425560       6        1   5.879003
21     -57.838154  164.474686       7        1   4.990592
8       67.619514  224.781052       8        1   4.929637
9      123.898613  165.483109       9        1   4.367870
26     -47.922863  238.327515      10        1   4.017570
1     -119.120422  215.052490      11        1   2.433880
4     -182.738937  106.019600      12        1   2.340949
31     -33.739548 -231.572327      13        1   1.297629
7       36.145504   54.118034      14        1   0.991517
29     113.255959   31.825455      15        1   0.794889
10    -180.028168  -62.060635      16        1   0.730486
19    -160.286270   10.950666      17        1   0.646041
17    -157.692627 -134.934372      18        1   0.615349
18     -31.333134   27.821592      19        1   0.573571
6      101.117378 -158.200470      20        1   0.570072
2     -105.471519  -43.905102      21        1   0.553596
34     -92.656006   19.148350      22        1   0.549550
11     145.094803  -83.416481      23        1   0.549233
27      19.017324   -4.349158      24        1   0.529197
13     -92.798775 -170.685135      25        1   0.525627
23      20.958708 -121.015335      26        1   0.524722
33     -33.615833 -148.007629      27        1   0.513823
16       4.648369  -65.861115      28        1   0.495327
14     108.237701  -29.963877      29        1   0.491467
12      74.995766  -98.689514      30        1   0.476406
15      29.249680 -180.817337      31        1   0.474936
25     -96.176323 -102.322731      32        1   0.465200
22     -43.377804  -31.362602      33        1   0.456947
20     -43.336403  -88.516701      34        1   0.449281
0       54.477737  -44.937836      35        1   0.446763, topic_info=            Term         Freq        Total Category  logprob  loglift
14679    unknown   404.000000   404.000000  Default  30.0000  30.0000
2393        cell  1006.000000  1006.000000  Default  29.0000  29.0000
7176   influenza   563.000000   563.000000  Default  28.0000  28.0000
3678          de   257.000000   257.000000  Default  27.0000  27.0000
1330       assay   368.000000   368.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
9892   olfactory     5.032390    15.523783  Topic35  -5.5161   4.2844
3243        copi     2.628885     5.322280  Topic35  -6.1655   4.7056
1616        barr     3.211080     8.135005  Topic35  -5.9654   4.4813
11848     recall     2.943238    10.325772  Topic35  -6.0525   4.1558
12836     server     2.828272    13.047061  Topic35  -6.0924   3.8820

[1862 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
40        12  0.652030         10(1
110        1  0.337999           1a
110        5  0.037555           1a
110        7  0.187777           1a
110        8  0.037555           1a
...      ...       ...          ...
15427     35  0.474739           α1
15439     29  0.719876     β-lactam
15448     30  0.848295  θ-defensins
15458     34  0.848953           φ6
15463      1  0.951369           −1

[5274 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 6, 25, 33, 29, 31, 22, 9, 10, 27, 2, 5, 32, 8, 30, 11, 20, 18, 19, 7, 3, 35, 12, 28, 14, 24, 34, 17, 15, 13, 16, 26, 23, 21, 1])

### Document Topic distribution

In [140]:
doc_topic_dist = pd.DataFrame(lda_output)
doc_topic_dist.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34
0,0.003571,0.003571,0.003571,0.003571,0.152917,0.492620,0.003571,0.003571,0.003571,0.240181,0.003571,0.003571,0.003571,0.003571,0.003571,0.003571,0.003571,0.003571,0.003571,0.003571,0.003571,0.003571,0.003571,0.003571,0.003571,0.003571,0.003571,0.003571,0.003571,0.003571,0.003571,0.003571,0.003571,0.003571,0.003571
1,0.004595,0.004595,0.004595,0.004595,0.004595,0.004595,0.004595,0.004595,0.004595,0.004595,0.004595,0.004595,0.004595,0.053190,0.004595,0.004595,0.004595,0.088970,0.004595,0.004595,0.004595,0.604783,0.004595,0.004595,0.004595,0.004595,0.110626,0.004595,0.004595,0.004595,0.004595,0.004595,0.004595,0.004595,0.004595
2,0.006558,0.006558,0.159002,0.006558,0.006558,0.096186,0.006558,0.006558,0.006558,0.006558,0.006558,0.006558,0.006558,0.006558,0.006558,0.006558,0.006558,0.006558,0.006558,0.006558,0.006558,0.006558,0.006558,0.006558,0.006558,0.006558,0.006558,0.006558,0.006558,0.006558,0.006558,0.006558,0.534971,0.006558,0.006558
3,0.003357,0.003357,0.003357,0.003357,0.003357,0.011194,0.003357,0.003357,0.003357,0.003357,0.003357,0.003357,0.003357,0.057964,0.003357,0.003357,0.003357,0.003357,0.043189,0.003357,0.003357,0.003357,0.003357,0.003357,0.783583,0.003357,0.003357,0.003357,0.003357,0.003357,0.003357,0.003357,0.003357,0.003357,0.003357
4,0.002960,0.002960,0.002960,0.002960,0.002960,0.348009,0.002960,0.002960,0.002960,0.002960,0.002960,0.002960,0.002960,0.002960,0.002960,0.002960,0.002960,0.002960,0.002960,0.002960,0.002960,0.015413,0.002960,0.002960,0.002960,0.002960,0.002960,0.002960,0.002960,0.002960,0.511686,0.002960,0.033133,0.002960,0.002960


### Topic's keyword distribution

In [141]:
topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

# topic keyword matrix
df_topic_keywords = pd.DataFrame(lda_model.components_)

# columns are the words
df_topic_keywords.columns = tfidf_vectorizer.get_feature_names()

# rows are the topics
df_topic_keywords.index = topicnames

df_topic_keywords.head()

,(+)-strand,(+)rna,(13)c,(fda)-approved,(hiv)-1,(ifn)-induced,(ifn)-mediated,(ifn)-α,(ifn)-β,(ifn)-γ,(il)-1,(il)-10,(il)-1β,(il)-2,(il)-6,(lps)-induced,(mers)-cov,(nf)-κb,(pi3k)/akt,(rt)-pcr,(sars)-associated,(sars)-cov,(sars)-like,(tnf)-α,-PRON-,-b,-c,-independent,-negative,0–2,0–4,0–5,1(st,"1,000-fold",1-,1-day-old,1-week,1-week-old,1-year,10,...,γδ,δg,θ-defensin,θ-defensins,κb,μg,μg/kg,μg/l,μg/ml,μl,μm,μmol/l,σ1,φ6,χ(2,χ2,–1,−/−,−1,−2,−20,∼10,∼100,∼2,∼20,∼30,∼50,≤5,≥1,≥18,≥2,≥3,≥30,≥4,≥5,≥50,≥6,≥60,≥65,≥80
Topic0,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,...,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571
Topic1,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,1.750578,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.930640,0.028571,0.028571,0.028571,0.028571,0.028571,1.443500,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,...,0.028571,0.028571,0.028571,0.028571,2.351705,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571
Topic2,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,1.503743,0.028571,0.028571,...,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.126626,0.165480,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571
Topic3,2.230490,5.103857,0.028571,0.028571,1.404381,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,1.391583,0.028571,0.028571,1.348612,0.028571,0.028571,2.472713,1.133495,0.028571,0.810480,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.817966,0.028571,0.028571,0.028571,0.028571,0.028571,...,0.028571,0.028571,0.028571,0.028571,0.028571,1.810813,0.028571,0.028571,7.321516,0.028571,38.154311,0.028571,3.173779,0.028571,0.028571,0.028571,1.937953,0.028571,24.630739,2.515227,0.379870,0.028571,0.028571,0.028571,1.913380,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571
Topic4,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,4.418493,0.298317,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,...,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,4.737944,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,1.919368

### Top 15 keywords in each topic

In [142]:
def top_words(vectorizer, lda_model=lda_model, n_words=15):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = top_words(tfidf_vectorizer, lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_top_keywords = pd.DataFrame(topic_keywords)
df_topic_top_keywords.columns = ['Word '+str(i) for i in range(df_topic_top_keywords.shape[1])]
df_topic_top_keywords.index = ['Topic '+str(i) for i in range(df_topic_top_keywords.shape[0])]
df_topic_top_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,unknown,mtb,npc,ebv,biomaterial,8b,oct4,olfactory,ntr,socs,côte,bag,rok,uc,puerto
Topic 1,cell,ifn,signal,pathway,response,expression,innate,activation,interferon,apoptosis,autophagy,gene,infection,immune,induce
Topic 2,respirator,mask,n95,t.,gondii,leishmania,ptsd,emodin,fit,lupus,leishmaniasis,wear,lynx,parasite,toxoplasma
Topic 3,protein,virus,cell,rna,viral,replication,bind,activity,membrane,structure,domain,host,gene,fusion,coronavirus
Topic 4,assay,detection,sample,test,sensitivity,pcr,detect,rt-pcr,method,amplification,rapid,specificity,real-time,diagnostic,diagnosis
Topic 5,disease,virus,pathogen,model,human,new,review,system,approach,development,drug,method,host,infectious,research
Topic 6,transplant,recipient,hpv,transplantation,hsct,malaria,cmv,nec,pv,donor,plasmodium,stem,hematopoietic,remyelination,falciparum
Topic 7,calve,herd,bovine,calf,cattle,diarrhea,cow,milk,farm,dairy,cryptosporidium,colostrum,pregnancy,rotavirus,pregnant
Topic 8,vaccine,antibody,epitope,protein,neutralize,response,virus,recombinant,mouse,antigen,sars-cov,immune,mabs,human,peptide
Topic 9,respiratory,child,virus,infection,influenza,rsv,patient,pneumonia,viral,rhinovirus,tract,detect,sample,pathogen,hmpv


### Get relevant articles

In [148]:
def get_k_nearest_docs(doc_dist, k=5, lower=1950, upper=2020):
    relevant_time = df.publish_year.between(lower, upper)

    temp = doc_topic_dist[relevant_time]
    
    distances = temp.apply(lambda x: jensenshannon(x, doc_dist), axis=1)

    k_distances = distances[distances != 0].nsmallest(n=k)
    k_nearest = distances[distances != 0].nsmallest(n=k).index
    return k_nearest, k_distances

In [161]:
def relevant_articles(query, k=5):
    query = [query] if type(query) is str else query

    query_vectorized = tfidf_vectorizer.transform(query)
    topic_dist = pd.DataFrame(lda_model.transform(query_vectorized))

    recommended_idx, scores = get_k_nearest_docs(topic_dist.iloc[0], k)
    recommended = df.iloc[recommended_idx]
    for idx, score in enumerate(scores):
        print(f"Title: {recommended.iloc[idx].title}")
        print(f"Abstract: {recommended.iloc[idx].abstract}")
        print(f"URL: {recommended.iloc[idx].url}")
        print(f"Published: {recommended.iloc[idx].publish_time}")
        print("-"*50)

In [163]:
query = "Are there any inflammations that will occur in the lungs due to COVID-19?"
relevant_articles(query, k=3)

Title: CCR2 Plays a Protective Role in Rocio Virus–Induced Encephalitis by Promoting Macrophage Infiltration Into the Brain
Abstract: Rocio virus (ROCV) is a highly neuropathogenic mosquito-transmitted flavivirus responsible for an unprecedented outbreak of human encephalitis during 1975–1976 in Sao Paulo State, Brazil. Previous studies have shown an increased number of inflammatory macrophages in the central nervous system (CNS) of ROCV-infected mice, implying a role for macrophages in the pathogenesis of ROCV. Here, we show that ROCV infection results in increased expression of CCL2 in the blood and in infiltration of macrophages into the brain. Moreover, we show, using CCR2 knockout mice, that CCR2 expression is essential for macrophage infiltration in the brain during ROCV infection and that the lack of CCR2 results in increased disease severity and mortality. Thus, our findings show the protective role of CCR2-mediated infiltration of macrophages in the brain during ROCV infection